In [71]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Hisse Kodlarının Alınması

In [72]:
hisseler = []
url = "https://www.isyatirim.com.tr/tr-tr/analiz/hisse/Sayfalar/sirket-karti.aspx?hisse=FROTO"
r = requests.get(url)
print(r)

<Response [200]>


In [73]:
s = BeautifulSoup(r.text, "html.parser")
s1 = s.find("select",id="ddlAddCompare")
c1 = s1.findChild("optgroup").findAll("option")

In [74]:
for a in c1:
    hisseler.append(a.string)

print(hisseler)

['ACSEL', 'ADEL', 'ADESE', 'AEFES', 'AFYON', 'AGESA', 'AGHOL', 'AGYO', 'AKBNK', 'AKCNS', 'AKENR', 'AKFGY', 'AKGRT', 'AKMGY', 'AKSA', 'AKSEN', 'AKSGY', 'AKSUE', 'AKYHO', 'ALARK', 'ALBRK', 'ALCAR', 'ALCTL', 'ALGYO', 'ALKA', 'ALKIM', 'ALMAD', 'ANELE', 'ANGEN', 'ANHYT', 'ANSGR', 'ARASE', 'ARCLK', 'ARDYZ', 'ARENA', 'ARMDA', 'ARSAN', 'ARZUM', 'ASELS', 'ASUZU', 'ATAGY', 'ATATP', 'ATEKS', 'ATLAS', 'ATSYH', 'AVGYO', 'AVHOL', 'AVOD', 'AVTUR', 'AYCES', 'AYDEM', 'AYEN', 'AYES', 'AYGAZ', 'AZTEK', 'BAGFS', 'BAKAB', 'BALAT', 'BANVT', 'BARMA', 'BASCM', 'BASGZ', 'BAYRK', 'BERA', 'BEYAZ', 'BFREN', 'BIMAS', 'BIOEN', 'BIZIM', 'BJKAS', 'BLCYT', 'BMSCH', 'BMSTL', 'BNTAS', 'BOBET', 'BOSSA', 'BRISA', 'BRKO', 'BRKSN', 'BRLSM', 'BRMEN', 'BRSAN', 'BRYAT', 'BSOKE', 'BTCIM', 'BUCIM', 'BURCE', 'BURVA', 'CANTE', 'CASA', 'CCOLA', 'CELHA', 'CEMAS', 'CEMTS', 'CEOEM', 'CIMSA', 'CLEBI', 'CMBTN', 'CMENT', 'CONSE', 'COSMO', 'CRDFA', 'CRFSA', 'CUSAN', 'DAGHL', 'DAGI', 'DAPGM', 'DARDL', 'DENGE', 'DERHL', 'DERIM', 'DESA', 'DE

# Bilançoların Çekilmesi

In [75]:
for i in hisseler:
    hisse = i
    tarihler = []
    yıllar = []
    donemler = []

    url1 = "https://www.isyatirim.com.tr/tr-tr/analiz/hisse/Sayfalar/sirket-karti.aspx?hisse="+hisse
    r1 = requests.get(url1)
    soup = BeautifulSoup(r1.text,"html.parser")
    secim = soup.find("select",id="ddlMaliTabloFirst")
    secim2 = soup.find("select",id="ddlMaliTabloGroup")

    try:
        cocuklar = secim.findChildren("option")
        grup = secim2.find("option")["value"]

        for i in cocuklar:
            tarihler.append(i.string.rsplit("/"))
        for j in tarihler:
            yıllar.append(j[0])
            donemler.append(j[1])
        if len(tarihler) >= 4 :
            parametreler = (
                ("companyCode",hisse),
                ("exchange","TRY"),
                ("financialGroup",grup),
                ("year1",yıllar[0]),
                ("period1",donemler[0]),
                ("year2",yıllar[1]),
                ("period2",donemler[1]),
                ("year3",yıllar[2]),
                ("period3",donemler[2]),
                ("year4",yıllar[3]),
                ("period4",donemler[3]))
            url2 = "https://www.isyatirim.com.tr/_layouts/15/IsYatirim.Website/Common/Data.aspx/MaliTablo"
            r2 = requests.get(url2,params=parametreler).json()["value"]
            veri = pd.DataFrame.from_dict(r2)
            veri.drop(columns=["itemCode","itemDescEng"],inplace=True)
        else:
            continue
    except AttributeError:
        continue
    del tarihler[0:4]
    tumveri = [veri]

    for _ in range(0,int(len(tarihler)+1)):
        if len(tarihler) == len(yıllar):
            del tarihler[0:4]
        else:
            yıllar = []
            donemler = []
            for j in tarihler:
                yıllar.append(j[0])
                donemler.append(j[1])

            if len(tarihler) >= 4 :
                parametreler2 = (
                    ("companyCode",hisse),
                    ("exchange","TRY"),
                    ("financialGroup",grup),
                    ("year1",yıllar[0]),
                    ("period1",donemler[0]),
                    ("year2",yıllar[1]),
                    ("period2",donemler[1]),
                    ("year3",yıllar[2]),
                    ("period3",donemler[2]),
                    ("year4",yıllar[3]),
                    ("period4",donemler[3]))

                r3 = requests.get(url2,params=parametreler2).json()["value"]
                veri2 = pd.DataFrame.from_dict(r3)
                try:
                    veri2.drop(columns=["itemCode","itemDescEng","itemDescTr"],inplace=True)
                    tumveri.append(veri2)
                except KeyError:
                    continue
    veri3 = pd.concat(tumveri, axis=1)
    baslık = ["Bilanço"]

    for i in cocuklar:
        baslık.append(i.string)

    baslıkfark = len(baslık)-len(veri3.columns)

    if baslıkfark != 0:
        del baslık[-baslıkfark:]

    veri3.set_axis(baslık, axis=1,inplace=True)
    veri3[baslık[1:]] = veri3 [baslık[1:]].astype(float)
    veri3.fillna(0,inplace=True)
    ### Excel sheets !!!
    veri3.to_excel(r"C:/Users/lenovo/PycharmProjects/pythonProject3/Data/Bilançolar/{}.xlsx".format(hisse),index=False)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))